In [45]:
import pandas as pd 
import numpy as np

In [46]:
mushroom = pd.read_csv('../data/mushroom.csv', header=None)

## 범주형 데이터 라벨링

In [47]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
for i in range(mushroom.shape[1]):
    mushroom.iloc[:,i] = encoder.fit_transform(mushroom.iloc[:,i])


In [48]:
mushroom.head()

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,1,5,2,4,1,6,1,0,1,4,...,2,7,7,0,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,...,2,7,7,0,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,...,2,7,7,0,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,...,2,7,7,0,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,...,2,7,7,0,2,1,0,3,0,1


In [49]:
X = mushroom.iloc[:,1:]
y = mushroom.iloc[:,0]

## 훈련, 검증, 테스트 셋나누기 

In [50]:
from sklearn.model_selection import train_test_split
X_trainval, X_test, y_trainval, y_test=  train_test_split(X, y, random_state=0)
X_train, X_valid, y_train, y_valid = train_test_split(X_trainval, y_trainval, random_state=1)  
print('훈련 세트의 크기:{}\n검증 세트의 크기:{}\n테스트 세트의 크기:{}'.format(X_train.shape[0], X_valid.shape[0], X_test.shape[0]))

훈련 세트의 크기:4569
검증 세트의 크기:1524
테스트 세트의 크기:2031


## feature 정규화

In [51]:
X_train = np.log1p(X_train)
X_test = np.log1p(X_test)
X_trainval = np.log1p(X_trainval)

## grid_search - SVC

In [52]:
from sklearn.svm import SVC
from tqdm import tqdm_notebook

best_score = 0
for gamma in tqdm_notebook([0.001, 0.01, 0.1, 1, 10, 100]):
    for C in [0.001, 0.01, 0.1, 1, 10, 100]:
        #매개변수의 각 조합에 대해SVC를 훈련시킵니다
        svm=SVC(gamma=gamma, C=C).fit(X_train, y_train)
        score = svm.score(X_valid, y_valid)
        if score > best_score:
            best_score = score
            best_param = {'C':C, 'gamma':gamma}
print('best score ==> {:.3f}'.format(best_score))
print('best parameter ==> {}'.format(best_param))


best score ==> 0.850
best parameter ==> {'C': 100, 'gamma': 0.001}


In [53]:
#훈련세트와 검증세트를 합쳐 모델을 다시 만든후
#테스트 세트를 사용해 평가한다
svm=SVC(**best_param).fit(X_trainval, y_trainval)
test_score=svm.score(X_test,y_test)
pred_svm = svm.predict(X_test)
print('최적 매개변수 점수 ==> {:.3f}'.format(best_score))
print('최적 매개변수 ==> {}'.format(best_param))
print('최적 매개변수에서 test ==> {:.3f}'.format(test_score))

최적 매개변수 점수 ==> 0.850
최적 매개변수 ==> {'C': 100, 'gamma': 0.001}
최적 매개변수에서 test ==> 0.955


## 최적 매개변수에서의 SVC score, classfication_report

In [54]:
from sklearn.metrics import accuracy_score
print('SVC 모델의 accuracy score:{:.2f}'.format(accuracy_score(y_test, pred_svm)))

SVC 모델의 accuracy score:0.95


In [55]:
from sklearn.metrics import classification_report
print('SVC 더미 모델의 classification_report :\n ',classification_report(y_test, pred_svm, target_names =['독', '식용']))

SVC 더미 모델의 classification_report :
               precision    recall  f1-score   support

          독       0.95      0.96      0.96      1061
         식용       0.96      0.95      0.95       970

avg / total       0.95      0.95      0.95      2031



## KNeighborsClassifier-accuracy_score

In [56]:
from sklearn.neighbors import KNeighborsClassifier
reg = KNeighborsClassifier(n_neighbors = 10).fit(X_train, y_train)
pred_reg = reg.predict(X_test)
print('KNN기반 모델의 accuracy :{:.2f}'.format(accuracy_score(y_test, pred_reg)))

KNN기반 모델의 accuracy :1.00


## KNeighborsClassifier-classification_report

In [69]:
from sklearn.metrics import classification_report
print('빈도 기반 더미 모델의 classification_report:\n ',classification_report(y_test, pred_reg, target_names =['독', '식용']))


빈도 기반 더미 모델의 classification_report:
               precision    recall  f1-score   support

          독       1.00      1.00      1.00      1061
         식용       1.00      1.00      1.00       970

avg / total       1.00      1.00      1.00      2031



## grid_search로 최적 매개변수 보기

In [66]:
from sklearn.model_selection import GridSearchCV

param_grid = {'n_neighbors':range(1,20)}
grid_search=GridSearchCV(KNeighborsClassifier(),param_grid,cv=5,return_train_score=True)
grid_search.fit(X_train,y_train)
print("트레인 세트 점수 : {:.2f}".format(grid_search.score(X_train,y_train)))
print("테스트 세트 점수 : {:.2f}".format(grid_search.score(X_test,y_test)))

print("최적 매개변수:",grid_search.best_params_)
print("최고 교차 검증 점수:",grid_search.best_score_)

print("최고 성능 모델:",grid_search.best_estimator_)

트레인 세트 점수 : 1.00
테스트 세트 점수 : 1.00
최적 매개변수: {'n_neighbors': 1}
최고 교차 검증 점수: 1.0
최고 성능 모델: KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=1, p=2,
           weights='uniform')


## RandomForestClassifier-accuracy_score

In [58]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators = 10)
forest.fit(X_train, y_train)
pred_forest = forest.predict(X_test)
print('랜덤 포레스트기반 모델의 accuracy :{:.2f}'.format(accuracy_score(y_test, pred_forest)))

랜덤 포레스트기반 모델의 accuracy :1.00


## RandomForestClassifier-classification_report

In [68]:
from sklearn.metrics import classification_report
print('랜덤 포레스트기반 기반 더미 모델의 classification_report:\n ',classification_report(y_test, pred_forest, target_names =['독', '식용']))


랜덤 포레스트기반 기반 더미 모델의 classification_report:
               precision    recall  f1-score   support

          독       1.00      1.00      1.00      1061
         식용       1.00      1.00      1.00       970

avg / total       1.00      1.00      1.00      2031



## grid_search로 최적 매개변수 보기

In [67]:
param_grid = {'n_estimators':range(1,20)}
grid_search=GridSearchCV(RandomForestClassifier(),param_grid,cv=5,return_train_score=True)
grid_search.fit(X_train,y_train)
print("트레인 세트 점수 : {:.2f}".format(grid_search.score(X_train,y_train)))
print("테스트 세트 점수 : {:.2f}".format(grid_search.score(X_test,y_test)))

print("최적 매개변수:",grid_search.best_params_)
print("최고 교차 검증 점수:",grid_search.best_score_)
print("최고 성능 모델:",grid_search.best_estimator_)

트레인 세트 점수 : 1.00
테스트 세트 점수 : 1.00
최적 매개변수: {'n_estimators': 5}
최고 교차 검증 점수: 1.0
최고 성능 모델: RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=5, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
